In [1]:
!pip install qiskit qiskit-aer --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.5 MB/s eta 0:00:00


In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Initialize simulator
sim = AerSimulator()

# Helper function to run and display results
def run_circuit(description, qc):
    compiled = transpile(qc, sim)
    result = sim.run(compiled, shots=1024).result()
    counts = result.get_counts()
    print(f"\n--- {description} ---")
    print(qc.draw())
    print("Measurement Counts:", counts)
    return counts

# --- 1️⃣ Original Circuit ---
qc1 = QuantumCircuit(2, 2)
qc1.h(0)
qc1.cx(0, 1)
qc1.cz(0, 1)
qc1.swap(0, 1)
qc1.measure_all()
counts1 = run_circuit("Original Circuit (H on q0, CX(0,1), CZ(0,1), SWAP)", qc1)

# --- 2️⃣ Control/Target Variation ---
qc2 = QuantumCircuit(2, 2)
qc2.h(0)
qc2.cx(1, 0)   # control=1, target=0
qc2.cz(1, 0)   # control=1, target=0
qc2.swap(0, 1)
qc2.measure_all()
counts2 = run_circuit("Control/Target Variation (CX(1,0), CZ(1,0))", qc2)

# --- 3️⃣ Gate Removal (remove SWAP) ---
qc3 = QuantumCircuit(2, 2)
qc3.h(0)
qc3.cx(0, 1)
qc3.cz(0, 1)
# qc3.swap(0, 1)  # Removed SWAP
qc3.measure_all()
counts3 = run_circuit("Gate Removal (SWAP removed)", qc3)

# --- 4️⃣ Add Another Hadamard on q1 ---
qc4 = QuantumCircuit(2, 2)
qc4.h(0)
qc4.h(1)       # Additional Hadamard on q1
qc4.cx(0, 1)
qc4.cz(0, 1)
qc4.swap(0, 1)
qc4.measure_all()
counts4 = run_circuit("Added Hadamard on q1 before CNOT", qc4)

# --- Visualization ---
plot_histogram(
    [counts1, counts2, counts3, counts4],
    legend=[
        "Original",
        "Control/Target Flipped",
        "SWAP Removed",
        "Extra H on q1"
    ],
    title="Comparison of Multi-Qubit Gate Variations"
)
plt.show()



--- Original Circuit (H on q0, CX(0,1), CZ(0,1), SWAP) ---
        ┌───┐            ░ ┌─┐   
   q_0: ┤ H ├──■───■──X──░─┤M├───
        └───┘┌─┴─┐ │  │  ░ └╥┘┌─┐
   q_1: ─────┤ X ├─■──X──░──╫─┤M├
             └───┘       ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 
Measurement Counts: {'11 00': 529, '00 00': 495}

--- Control/Target Variation (CX(1,0), CZ(1,0)) ---
        ┌───┐┌───┐       ░ ┌─┐   
   q_0: ┤ H ├┤ X ├─■──X──░─┤M├───
        └───┘└─┬─┘ │  │  ░ └╥┘┌─┐
   q_1: ───────■───■──X──░──╫─┤M├
                         ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 
Measurement Counts: {'10 00': 529, '00 00': 495}

--- Gate Removal (SWAP removed) ---
        ┌───┐         ░ ┌─┐   
   q_0: ┤ H ├──■───■──░─┤M├───
        └───┘┌─┴─┐ │  ░ └╥┘┌─┐
   q_1: ─────┤ X ├─■──░──╫─┤M├
             └──